NB: Scriptet bruges på eget ansvar 
 
Generelt script der importerer overfladetemperaturer fra sommermånederne. Der bliver lavet et gennemsnit fra hver sommer som anvendes som et 'årsgennemsnit'. 
Der laves zonal statistics på en tabel man selv kan vælge 

Der hentes data direkte fra NASA gennem Earth Engine (ee). Det er et pythonplugin fra Google og det kræver autorisation med en jsonfil. Hvis authentication fejler, er det højst sandsynligt fordi stien til json-filen ikke er korrekt. 

Det hele er automatiseret og der kan derfor være satellitbilleder med fejl eller mangler, så det er altid en god idé at validere resultaterne.

Google har nogle begrænsning ift hvor meget man kan udnytte deres servere, så hvis der er mange features i ens inputtabel kan man godt risikere at scriptet fejler. 


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

import ee
import geemap

# Initialize GEE/GCLOUD
email = '[...]@gmail.com'
keyfile= r"c:/dokumenter/[].json"
credentials = ee.ServiceAccountCredentials(email=email, key_file=keyfile)
ee.Initialize(credentials)

Definition af tabeller der skal bruges som input og navne på outputtabeller. 
Det eneste, der skal rettes er variablerne: 

Inputtabel: tabel i db man gerne vil anvende som ROIS (områder der skal analyseres)
tabelnavnLST + tabelnavnvegdaekke: navn på outputtabeller til hhv LST/UHI og vegetationsdække og resultatet af analysen som ender i det skema, der defineres i 'pgskema' nedenfor


In [3]:
# ***************************************************
#    Defintion af områder der skal analyseres
# ***************************************************
# Navn på tabel i db, der skal undersøges 
Inputtabel = 'trafik.vejflade'

# Navn på tabel, der anvendes som ROI til import af image collections 
Extenttabel = 'admin.kommunegraense'

# Navngivning til outputdata
pgskema = ''
tabelnavnLST = ''
tabelnavnvegdaekke = ''


# Sti til geopackage der indeholder alle resultater
geopacksamlet = r"\samletgeopack.gpkg"

# Connections til db
db_connection_url = "postgresql://"
con = create_engine(db_connection_url)


#Områder
rois = """SELECT * from """+Inputtabel +""" where wkb_geometry is not null  and ST_isvalid(wkb_geometry)is true"""
roisgdf = gpd.GeoDataFrame.from_postgis(rois, con, geom_col='wkb_geometry')             # Anvendes i pandas/geopandas funktioner
roisDF = geemap.geopandas_to_ee(roisgdf)                                                # Anvendes i ee funktioner

# Extents til imagecollections
extents = """SELECT st_union(wkb_geometry) as wkb_geometry from """+Extenttabel+""" where wkb_geometry is not null  and ST_isvalid(wkb_geometry)is true"""
extentsDF = gpd.GeoDataFrame.from_postgis(extents, con, geom_col='wkb_geometry')        # Anvendes i pandas/geopandas funktioner
roi = geemap.geopandas_to_ee(extentsDF)                                                 # Anvendes i ee funktioner

Definition af funktioner 
Der er defineret funktioner til at rense satellitbilleder for dårlige pixels samt til at definere de bånd der anvendes i analysen. 
Funktion annualsummerzonals er den der beregner zonal statistics på de områder, man har valgt som Inputtabel. 

In [4]:
#*********************************************
#               Funktioner
#*********************************************
def createBands(image):
    # Beregner bands der anvendes til analysen
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI').reproject('EPSG:25832', None, 30)
    thermal = image.select('ST_B10').multiply(0.00341802).add(149.0).subtract(272.15).rename('lst')
    image = image.addBands(ndvi).addBands(thermal).set('system_time_start', image.get('system:time_start')).set('Date', ee.Date(image.get('system:time_start')))
    return image

def createBandsL5(image):
    ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI').reproject('EPSG:25832', None, 30)
    thermal = image.select('ST_B6').multiply(0.00341802).add(149.0).subtract(272.15).rename('lst')
    image = image.addBands(ndvi).addBands(thermal).set('system_time_start', image.get('system:time_start')).set('Date', ee.Date(image.get('system:time_start')))
    return image

  
def sentinelNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).reproject('EPSG:25832', None, 10)
    reclass = ndvi.rename('sentinelndvi')
    return reclass.set('system:time_start', image.get('system:time_start')).copyProperties(image)

def maskS2clouds(image):
    qa = image.select('QA60')
    # // Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # // Both flags should be set to zero, indicating clear conditions.
    mask1 = qa.bitwiseAnd(cloudBitMask).eq(0)
    mask2 = qa.bitwiseAnd(cirrusBitMask).eq(0)
    
    return image.updateMask(mask1).updateMask(mask2).divide(10000).copyProperties(image)


def cleanUpAllLandsat(image):
  #Develop masks for unwanted pixels (fill, cloud, cloud shadow).
  qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
  saturationMask = image.select('QA_RADSAT').eq(0)
  # Replace original bands with scaled bands and apply masks.
  return image.updateMask(qaMask).updateMask(saturationMask)

# Funktion til at lave zonal stats med UHI 
def annualsummerzonals(startaar,slutaar,imgseries,regions,tablename,schemaname):
    
    print(f"Laver zonals for {tablename}")
    master_gdfmerged_landsatfunc = gpd.GeoDataFrame()
    aarsliste = list(range(startaar, slutaar))
    for aar in aarsliste:
        
        aarstal = str(aar)
        
        landsatimg = imgseries.select('lst','NDVI').filterDate(aarstal+'-06-01',aarstal+'-12-31')
        if landsatimg.size().getInfo() == 0:
            print(f"Ingen billeder i {aarstal}")
            continue  # Skip this iteration and proceed to the next year
        
        inputlandsat =landsatimg.mean() 
        
        dictio = inputlandsat.select('lst','NDVI').reduceRegions(regions ,ee.Reducer.mean(),scale = 30).set('dato', image.get('Date')).set('system_index_t', image.get('system:index'))
        gdf_landsat = geemap.ee_to_geopandas(dictio)
        gdf_landsat['aar'] = aar 
        gdf_landsat = gdf_landsat.set_crs(4326)
        gdf_landsat = gdf_landsat.to_crs(25832)
        
        gdf_landsat = gdf_landsat.rename(columns={'lst': 'uhi'})
        gdf_landsat = gdf_landsat.rename(columns={'NDVI': 'ndvi'})

        master_gdfmerged_landsatfunc = pd.concat([master_gdfmerged_landsatfunc, gdf_landsat])
        

    master_gdfmerged_landsatfunc.to_postgis(name = tablename, schema = schemaname, con = con,if_exists ='replace')
    print(f"Zonal for {tablename} er lavet \n")
    return master_gdfmerged_landsatfunc

Image collections 
Her defineres de collections der anvendes 

In [5]:
#Definition af imagecollections:
landsat5_summer = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterDate('1984-06-01', '2013-12-31').filter(ee.Filter.calendarRange(6, 8, 'month')).filter(ee.Filter.lt('CLOUD_COVER_LAND', 20)).filterBounds(roi).map(createBandsL5).map(cleanUpAllLandsat)
landsat8_summer = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2013-06-01','2030-12-31').filter(ee.Filter.calendarRange(6, 8, 'month')).filter(ee.Filter.lt('CLOUD_COVER_LAND', 20)).filterBounds(roi).map(createBands).map(cleanUpAllLandsat)
landsat9_summer = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate('2021-06-01','2030-12-31').filter(ee.Filter.calendarRange(6, 8, 'month')).filter(ee.Filter.lt('CLOUD_COVER_LAND', 20)).filterBounds(roi).map(createBands).map(cleanUpAllLandsat)

landsatcol1 = landsat8_summer.merge(landsat9_summer)
landsatcol = landsat5_summer.merge(landsatcol1)


sentinel_2017 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2017-06-01','2017-12-31').filter(ee.Filter.calendarRange(6,8,'month')).filterBounds(roi).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',15)).map(sentinelNDVI).mean()
sentinel_2018 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2018-06-01','2018-12-31').filter(ee.Filter.calendarRange(6,8,'month')).filterBounds(roi).map(maskS2clouds).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(sentinelNDVI).mean()
sentinel_2019 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2019-06-01','2019-12-31').filter(ee.Filter.calendarRange(6,8,'month')).filterBounds(roi).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',15)).map(sentinelNDVI).mean()
sentinel_2020 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2020-06-01','2020-12-31').filter(ee.Filter.calendarRange(6,8,'month')).filterBounds(roi).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',15)).map(sentinelNDVI).mean()
sentinel_2021 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2021-06-01','2021-12-31').filter(ee.Filter.calendarRange(6,8,'month')).filterBounds(roi).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',15)).map(sentinelNDVI).mean()
sentinel_2022 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2022-06-01','2022-12-31').filter(ee.Filter.calendarRange(6,8,'month')).filterBounds(roi).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',15)).map(sentinelNDVI).mean()
sentinel_2023 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2023-06-01','2023-12-31').filter(ee.Filter.calendarRange(6,8,'month')).filterBounds(roi).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)).map(sentinelNDVI).mean()


Beregning af vegetationsdække. 

Der laves en beregning af vegetationsdække, der er defineret som alle pixels med ndvi > 0,3. De konverteres til vektor og skrives til en geodataframe, der laves zonal statistics på med roder og det rå vegetationsdække eksporteres til geopackage

In [ ]:
listsenti = [sentinel_2017,sentinel_2018,sentinel_2019,sentinel_2020,sentinel_2021,sentinel_2022,sentinel_2023]
listsentistring = ['sentinel_2017','sentinel_2018','sentinel_2019','sentinel_2020','sentinel_2021','sentinel_2022','sentinel_2023']

list_varibale_names = []

master_gdfmerged = gpd.GeoDataFrame()

sentinel_gdfs = {}
for idx,image in enumerate(listsenti): 
    senti = image.select('sentinelndvi').gt(0.3).selfMask()

    vectors = senti.reduceToVectors(
            geometry=roi,
            scale=10,
            crs ='EPSG:25832',
            geometryType='polygon',
            eightConnected=True,
            labelProperty='NDVI'
            
        )

    geometryDissolve = vectors.union(10)
    layername = [name for name, value in locals().items() if value is image][0]

    gdf = geemap.ee_to_geopandas(geometryDissolve)
    variable_name = f"{layername}_gdf"
    sentinel_gdfs[variable_name] = gdf
    
    
    list_varibale_names.append(variable_name)

    gdf.to_file(geopacksamlet,layer=layername, driver='GPKG')

for layer in listsentistring: 
    try:
        vegvector = gpd.read_file(geopacksamlet, layer=layer)  

        vegvector =vegvector.to_crs(25832)
        
        aarstal = layer.replace("sentinel_", "")
        vegvector['aar'] = aarstal

        merged = gpd.overlay(roisgdf, vegvector, how='intersection')
        merged['areal_vegetation'] = merged.area

        merged['andel_vegetation'] = merged['areal_vegetation']/merged['rodearealberegnet']*100

        master_gdfmerged = master_gdfmerged.append(merged, ignore_index=True)
        # master_gdfmerged = gpd.GeoDataFrame(pd.concat(gdfmerged, ignore_index=True))
    except ValueError as e:
        print(f"Layer '{layer}' does not exist or could not be loaded. Error: {e}")
        continue

Kolonnejonglering af geodataframe og eksport til db og geopackage

In [ ]:
# Samler geodataframe med roder 
rodermveg = master_gdfmerged.merge(roisgdf, on='rode_nr', suffixes=('_veg', '_rode'))
# # Keep only the geometry from the right table
rodermveg['geometry'] = rodermveg['wkb_geometry']
rodermveg['rodenavn'] = rodermveg['rodenavn_veg']

# Drop unnecessary columns
columns_to_drop = ['wkb_geometry']
rodermveg.drop(columns=columns_to_drop, inplace=True)

# Skriver data til hhv. geopackage og db
rodermveg.to_file(geopacksamlet,layer='rodermedveg', driver='GPKG')
rodermveg.to_postgis(name = tabelnavnvegdaekke, schema = pgskema, con = con,if_exists ='replace') 


Herunder køres funktionen med zonal statistics for roder, kvarterer, bydele og hele København. De forskellige kørsler ligger i fire sektioner. 

Det tager et sted mellem 40 og 60 minutter at køre det hele. 

In [ ]:
# Automatisk øvre grænse for år, så det nyeste data kommer med 
from datetime import datetime
year = int(datetime.today().strftime('%Y'))+1

# *** Roder ***
inputDFUHI = annualsummerzonals(1984,year,landsatcol,roisDF,tabelnavnLST,tabelnavnvegdaekke)
